In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.python.keras import applications
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from tensorflow.python.keras.optimizers import RMSprop
import numpy as np
import pandas as pd

In [2]:
#parameters
classes = 12
batch_size=128
train_total = 3783
validation_total = 967

In [3]:
model_resnet = ResNet50(include_top=False, weights = 'imagenet',input_shape=(224,224,3))

In [4]:
x = model_resnet.get_layer('avg_pool').output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [5]:
model = Model(inputs=model_resnet.input, outputs=x)

In [6]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        input_1[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 112, 112, 64)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [7]:
for layer in model_resnet.layers:
    layer.trainable= False

In [8]:
model.get_layer('res5a_branch2a').trainable= True
model.get_layer('bn5a_branch2a').trainable= True
model.get_layer('res5a_branch2b').trainable= True
model.get_layer('bn5a_branch2b').trainable= True
model.get_layer('res5a_branch2c').trainable= True
model.get_layer('res5a_branch1').trainable= True
model.get_layer('bn5a_branch2c').trainable= True
model.get_layer('bn5a_branch1').trainable= True
model.get_layer('res5b_branch2a').trainable= True
model.get_layer('bn5b_branch2a').trainable= True
model.get_layer('res5b_branch2b').trainable= True
model.get_layer('bn5b_branch2b').trainable= True
model.get_layer('res5b_branch2c').trainable= True
model.get_layer('bn5b_branch2c').trainable= True
model.get_layer('res5c_branch2a').trainable= True
model.get_layer('bn5c_branch2a').trainable= True
model.get_layer('res5c_branch2b').trainable= True
model.get_layer('bn5c_branch2b').trainable= True
model.get_layer('res5c_branch2c').trainable= True
model.get_layer('bn5c_branch2c').trainable= True

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(224,224),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 3820 images belonging to 12 classes.
Found 930 images belonging to 12 classes.


In [11]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-7),
             metrics=['accuracy'])

In [12]:
checkpointer = ModelCheckpoint(filepath='resnet50_fixed_bottom_save_best.hdf5', verbose=1, save_best_only=True, save_weights_only=True)


In [14]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=50,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=[checkpointer])

Epoch 1/50
Epoch 00000: val_loss improved from inf to 0.15075, saving model to resnet50_fixed_bottom_save_best.hdf5
178s - loss: 0.0914 - acc: 0.9690 - val_loss: 0.1508 - val_acc: 0.9505
Epoch 2/50
Epoch 00001: val_loss did not improve
93s - loss: 0.1032 - acc: 0.9615 - val_loss: 0.1531 - val_acc: 0.9484
Epoch 3/50
Epoch 00002: val_loss improved from 0.15075 to 0.13578, saving model to resnet50_fixed_bottom_save_best.hdf5
60s - loss: 0.0951 - acc: 0.9684 - val_loss: 0.1358 - val_acc: 0.9570
Epoch 4/50
Epoch 00003: val_loss did not improve
69s - loss: 0.0923 - acc: 0.9673 - val_loss: 0.1609 - val_acc: 0.9495
Epoch 5/50
Epoch 00004: val_loss improved from 0.13578 to 0.12515, saving model to resnet50_fixed_bottom_save_best.hdf5
69s - loss: 0.0855 - acc: 0.9712 - val_loss: 0.1252 - val_acc: 0.9570
Epoch 6/50


KeyboardInterrupt: 

In [30]:
model.save_weights('resnet50_bottom_fixed.hdf5')

In [13]:
model.load_weights('resnet50_from_scratch_save_best.hdf5')

In [14]:
model.save('resnet50_from_scratch_save_model.h5')

In [16]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(224,224),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [17]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [18]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_resnet_fixed_bottom.csv', encoding="utf8", index=False)

In [19]:
from IPython.display import FileLink
FileLink('submission_resnet_fixed_bottom.csv')

/home/janardhan/Documents/plant_seeding/submission_resnet_fixed_bottom.csv

In [ ]:
# PB LB: 0.93702